# Vaccination Outcomes of World...
In this notebook we will be doing EDA on the vaccine data of the different countries with their population v/s total vaccinations v/s people vaccinated mainly.

## The data is dated till 29 Janunary (Last updated)

# Necessary Imports

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = False)
import cufflinks as cf
cf.go_offline(True)

Reading the files csv data files.

In [ ]:
data=pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
data2=pd.read_csv('../input/covid-19/data/countries-aggregated.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr())

In [ ]:
data.corr()['total_vaccinations_per_hundred'].sort_values().plot(kind='bar')

In [ ]:
data.corr()['people_vaccinated_per_hundred'].sort_values().plot(kind='bar')

In [ ]:
data.corr()['daily_vaccinations_per_million'].sort_values().plot(kind='bar')

**Moreover we will be using plotly's choropleth map-graph which gives a clear and interactive understanding on the feautres by there corresponding place on map.
To read more about :-  [Plotly's Choropleth Graphs](https://plotly.com/python/choropleth-maps/)**

In [ ]:
fig=px.choropleth(data_frame=data,
                          locations=data['country'],
                          locationmode='country names',
                          color=data['daily_vaccinations_per_million'],
                          animation_frame=data['date'],
                          animation_group=data['vaccines'])
fig.update_layout(dict1={'title':'Daily Vaccinations Per Million'})

fig.show() 

**Now we will be using the plotly's scatter_geo graph for plotting the corresponding daily_vaccination numbers by thier place or continent in use.
To read more about :- [Plotly's scatter_geo graph](https://plotly.com/python/scatter-plots-on-maps/)**

In [ ]:
data.dropna(subset=['daily_vaccinations'],inplace=True)
s=data['date'].str.split('-',expand=True)
data['Year']=s[0]
data['Month']=s[1]
data['Date']=s[2]
fig1=px.scatter_geo(data,color='vaccines',locationmode="ISO-3",locations="iso_code",opacity=0.6,
                     hover_name="iso_code", size="daily_vaccinations",projection='conic equal area',animation_group ="iso_code",color_continuous_scale='blackbody',
                     animation_frame="Date",scope='world',symbol='vaccines',template="plotly_dark",title='Vaccination Count Over The World Over Time' )
fig1.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 400

fig1.update_geos(
    landcolor="green",
    oceancolor="blue",
    showocean=True,
    lakecolor="LightBlue"
)

fig1.update_traces(
    marker_coloraxis=None
)
fig1.show()

**We can clearly see that Pfizer/Biotech's impact in USA and Covaxin's impact on India is huge and it keeps on growing by the date increases . We can also see heavy impact of CNBG/Cinovac starting in China . Sinovac takes a sudden roll in Turkey on 14th and keeps on growing after that :)**

# Which vaccine is distributed in the world the most ??¶

In [ ]:
data.groupby('vaccines')['country'].count().iplot(title='Covid vaccines Count')

We see the Pfizer/BioNTech, AstraZeneca and Moderna is mostly used for vaccination.

In [ ]:
fig=px.choropleth(data_frame=data,
                          locations=data['country'],
                          locationmode='country names',
                          color=data['total_vaccinations_per_hundred'],
                          animation_frame=data['date'],
                          animation_group=data['vaccines'])
fig.update_layout(dict1={'title':'Total Vaccinations per hundred'})
fig.show() 

By seeing the below graph, we can estimate the impact of Pfizer, BioNTech and Moderna vaccine on the world.

In [ ]:
s=data.drop_duplicates(subset=['iso_code'])['vaccines'].apply(lambda x: x.split(','))
dic={}
for i in s:
    for j in i :
        if j[0]==' ':
            k= j[1:]
        elif j[-1]==' ':
            k=j[:-1]
        else:
            k=j
        if k not in dic :
            dic[k]=1
        else:
            dic[k]+=1
px.bar(x=list(dic.keys()),y=list(dic.values()),color=list(dic.keys()),template='plotly_dark',labels={'x':'Vaccine Name','y':'Total Count'})

In [ ]:
df = px.data.gapminder()
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year",
                     projection="natural earth")
fig.show()

From this we cans see that Pfizer is the vaccine most spread in the world in Januray 2021 followed by Moderna and Sinovac :)

In [ ]:
data.groupby('country')['total_vaccinations_per_hundred'].mean().sort_values()[40:].iplot(title='Top 20 countries based on Total Vaccinations Per Hundred')
data.groupby('country')['daily_vaccinations_per_million'].mean().sort_values()[40:].iplot(title='Top 20 countries based on Daily Vaccinations Per Million ')

# Change in death rate after introduction of Vaccine
Now we will be using the sencond dataset regarding the deaths, recoverings and confirmed cases.

In [ ]:
arr=[]
index1=data.groupby(['country','date']).count().index
index2=data2.groupby(['Country','Date']).count().index
for i in index2:
    if i in index1:
        arr.append(1)
    else:
        arr.append(0)
data2['Vaccine_is_there']=arr
data2.head()

In [ ]:
t=data2.groupby(['Country']).sum()
cons=t[t['Vaccine_is_there']==0].index
dan=data2[~data2['Country'].isin(cons)]

The below graph gives us the freedom to choose a country of out choice from the dropdown and see the corresponding outcomes.

In [ ]:
data2['Date']=pd.to_datetime(data2['Date'])
dfs=list(data2.groupby("Country"))
first_title = dfs[0][0]
traces = []
buttons = []
for i,d in enumerate(dfs):
    visible=[False]*len(dfs)
    visible[i]=True
    name=d[0]
    p=d[1][d[1]['Vaccine_is_there']==0]
    n=d[1][d[1]['Vaccine_is_there']==1]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=p['Date'],y=p['Deaths'].values))
    fig.add_trace(go.Scatter(x=n['Date'],y=n['Deaths'].values))
    
    traces.append(
    fig.update_traces(visible=True if i==0 else False).data[0])
    buttons.append(dict(label=name,
                        method="update",
                        args=[{"visible":visible},
                              {"title":f"{name}"}]))

updatemenus = [{'active':0, "buttons":buttons}]
shapes=[({'type': 'line',
               'xref': 'x',
               'yref': 'y',
               'x0': '2020-12-29' ,
               'y0': 0,
               'x1': '2020-12-29',
               'y1': 3000})]
fig = go.Figure(data=traces,
                 layout=dict(updatemenus=updatemenus,shapes=shapes,template='plotly_dark'))
fig.update_layout(title=first_title, title_x=0.5)
fig.show()

### After the white line the vaccine was introduced :) You can see the results for different countries with using the drop down menu.
Currently looking for a solution for the drop down bug. Please comment below if you have any.

# Please Upvote and Comment below if you liked this notebook.
### Any suggestiongs are accepted and would be happy to listen to your feedback.😊

# Thank You